<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

## Response Processing
*The core of the Datastar integration involves replacing FastHTML's response processing to generate Server-Sent Events (SSE) instead of traditional HTML responses.*

This function processes FastHTML responses but removes HTMX-specific headers that aren't needed for Datastar:

In [57]:
from fasthtml.common import *
from starlette.testclient import TestClient

In [58]:
# Test with actual FastHTML app and TestClient
app = FastHTML()

@app.get("/test-headers")
def test_headers():
    return Div("Content"), HttpHeader("X-Custom", "test-value")

client = TestClient(app)
response = client.get("/test-headers")
assert response.headers["X-Custom"] == "test-value"

In [60]:
# Test fragment vs full page detection
app = FastHTML(title="Test App")

In [61]:
@app.get("/fragment")
def fragment():
    return Div(id="content")("Fragment content")

In [62]:
client = TestClient(app)

Regular request gets full page

In [63]:
response = client.get("/fragment")
assert "<title>Test App</title>" in response.text
assert "<div id=\"content\">Fragment content</div>" in response.text

Datastar request gets fragment only

In [64]:
response = client.get("/fragment?datastar=true")
# This would be just the fragment in a real implementation

Test JSON responses become signals

In [66]:
app = FastHTML()

@app.get("/signals")
def signals():
    return {"counter": 42, "message": "hello"}

client = TestClient(app)
response = client.get("/signals")
assert response.json() == {"counter": 42, "message": "hello"}

## Route Enhancement

In [0]:
#| echo: false
#| output: asis
show_doc(FastHTML.route)

---

### FastHTML.route

>      FastHTML.route (path:str=None, methods=None, name=None,
>                      include_in_schema=True, body_wrap=None, patch_mode=None,
>                      selector=None, use_view_transition=False)

*Enhanced route decorator with server-driven Datastar patch options*

Update HTTP method shortcuts

In [68]:
for method in all_meths:
    setattr(FastHTML, method, partialmethod(FastHTML.route, methods=method))

In [69]:
# Test route with patch options
app = FastHTML()

@app.get("/append-item", patch_mode="append", selector="#list")
def append_item():
    return Li("New item")

# Verify the function has Datastar attributes
assert hasattr(append_item, '_datastar_mode')
assert append_item._datastar_mode == "append"
assert append_item._datastar_selector == "#list"

## Endpoint Processing

## Monkey Patch Application